###### We used the CDS API to download ERA5 reanalysis data by specifying key parameters such as temperature and dew point, along with the desired time range and spatial coverage. After registering on the Copernicus Climate Data Store (CDS) and configuring the API, we submitted a request to retrieve the data in NetCDF format. This allowed us to efficiently access high-resolution climate information, enabling a detailed analysis of temperature trends and extreme heat events over time. The automated download process streamlined data retrieval, making it easier to work with large climate datasets.######

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install cdsapi geopandas rioxarray xarray pandas netCDF4

In [ ]:
#! pip install "cdsapi>=0.7.2"

In [ ]:
import cdsapi
import xarray as xr
import geopandas as gpd
import rioxarray

In [ ]:
# Extracting sample temperature 2m data for one year
import cdsapi

dataset = "derived-era5-single-levels-daily-statistics"
request = {
    "product_type": "reanalysis",
    "variable": ["2m_temperature"],
    "year": "2001",
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "daily_statistic": "daily_mean",
    "time_zone": "utc+00:00",
    "frequency": "6_hourly",
    "area": [35, 68, 6, 97]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

In [ ]:
!pip install pandas numpy plotly


In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
from scipy.stats import pearsonr
import glob
import plotly.express as px
import numpy as np

import plotly.graph_objects as go
from scipy import stats


In [ ]:
ERA_5_INDIA_DATA_DIR = "/content/drive/MyDrive/ERA_5_INDIA_DATA_DIR"

In [ ]:
#Computes relative humidity (RH) from temperature (t) and dewpoint temperature (d).
#Uses an empirical equation to estimate Wet Bulb Temperature (WBT).
#Returns WBT.

def calculate_wbt(t, d):
    """Approximate Wet Bulb Temperature (WBT) from temperature and dewpoint."""
    rh = 100 * (np.exp((17.625 * d) / (243.04 + d)) / np.exp((17.625 * t) / (243.04 + t)))  # Relative Humidity
    wbt = t * np.arctan(0.151977 * (rh + 8.313659)**0.5) + \
          np.arctan(t + rh) - np.arctan(rh - 1.676331) + \
          0.00391838 * rh**1.5 * np.arctan(0.023101 * rh) - 4.686035
    return wbt

In [ ]:
# Reads a GeoJSON file containing state boundaries for India.
# This is used to map WBT values to specific states.
# Iterates through years (2000 to 2023).
# For each year, iterates through months (Jan-Dec).
# Processes WBT data monthly.
# Defines file paths for temperature (t2m) and dewpoint temperature (d2m) for a given month.
# Reads these NetCDF files using xarray
# ERA5 temperature data is in Kelvin, so we subtract 273.15 to convert to Celsius.
# Uses the calculate_wbt function to compute WBT for each grid point in the dataset.
# Converts xarray dataset into a pandas DataFrame.
# Renames columns for clarity.
# Creates geospatial points for each data location using longitude and latitude.
# Converts DataFrame to GeoDataFrame for spatial operations.
# Performs a spatial join to assign each WBT data point to the corresponding state.
# Extracts year, month, and day from the time column for grouping and analysis.
# Groups by year, month, day, and state (st_nm).

# Computes mean values for temperature, dewpoint, and WBT.
# Concatenates monthly WBT data into a single yearly DataFrame.
# Saves the results to a CSV file.
# Collects all yearly CSV files.
# Reads them into a list of DataFrames.
# Merges all years into a single CSV file.



# Load shapefile from GeoJSON
states_gdf = gpd.read_file("/content/drive/MyDrive/Indian_states/Indian_states.geojson")

for year in range(2000, 2024):
    year_wbt = []
    for month in range(1, 13):
        t2m_file_path = f"{ERA_5_INDIA_DATA_DIR}/max-1-hourly/{year}-{month}/2m_temperature_0_daily-max.nc"
        d2m_file_path = f"{ERA_5_INDIA_DATA_DIR}/max-1-hourly/{year}-{month}/2m_dewpoint_temperature_stream-oper_daily-max.nc"

        print(f"Processing {t2m_file_path}, {d2m_file_path}")

        # Load the netCDF files
        ds = xr.open_dataset(t2m_file_path)
        ds_d2m = xr.open_dataset(d2m_file_path)

        ds['t2m'] = ds['t2m'] - 273.15  # Convert to Celsius
        ds['d2m'] = ds_d2m['d2m'] - 273.15  # Convert to Celsius

        # Rename valid_time to time (if needed)
        if 'valid_time' in ds:
            ds = ds.rename({'valid_time': 'time'})

        # Apply WBT calculation
        ds['wbt'] = calculate_wbt(ds['t2m'], ds['d2m'])

        df = ds[['t2m', 'd2m', 'wbt']].to_dataframe().reset_index()
        df = df.rename(columns={'t2m': 'Temperature_C', 'd2m': 'Dewpoint_C', 'wbt': 'Wet_Bulb_Temperature_C'})

        # Convert DataFrame grid points to GeoDataFrame
        geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
        points_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

        # Perform spatial join to map points to states
        points_with_states = gpd.sjoin(points_gdf, states_gdf, how="inner", predicate="intersects")

        # Extract year, month and day from 'time' if it is a datetime object
        points_with_states['year'] = points_with_states['time'].dt.year
        points_with_states['month'] = points_with_states['time'].dt.month
        points_with_states['day'] = points_with_states['time'].dt.day

        # Group by year and state (using 'st_nm' for state names)
        year_wbt.append(points_with_states.groupby(['year', 'month', 'day', 'st_nm'])[['Temperature_C', 'Dewpoint_C', 'Wet_Bulb_Temperature_C']].mean().reset_index())

    # Concatenate all_wbt and save as a CSV file
    year_wbt_df = pd.concat(year_wbt, ignore_index=True)
    year_wbt_df.to_csv(f"{ERA_5_INDIA_DATA_DIR}/statewise_wbt_max_{year}_test.csv", index=False)

In [ ]:
# file path
# This CSV contains daily mean max temperature, dewpoint, and WBT for each state.

csv_files = glob.glob("/content/drive/MyDrive/ERA_5_INDIA_DATA_DIR/statewise_wbt_max_*_test.csv")

# Read and concatenate all CSVs
df_list = [pd.read_csv(file) for file in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# This final CSV contains daily records for every state over all years.
# Save the merged CSV
df_all.to_csv("/content/drive/MyDrive/ERA_5_INDIA_DATA_DIR/statewise_wbt_max_all_years.csv", index=False)

print("Merged CSV file saved successfully!")

# Each daily record is statewise, meaning temperature and dew point values are aggregated across grid points within each state.
# The final dataset provides a statewise daily time series from 2000 to 2023.
# All years are merged into a single statewise daily dataset.

"""Each daily record is statewise, meaning that:

    Temperature (t2m) values are aggregated across all grid points within each state.
    Dewpoint (d2m) values are also aggregated across all grid points within each state.
    Wet Bulb Temperature (WBT) is computed for each grid point, then aggregated statewise.

So for every state and day, the dataset contains:
 Mean temperature (°C) across the state
 Mean dewpoint (°C) across the state
 Mean wet bulb temperature (°C) across the state

These are saved daily in the statewise CSV files. """

##### Calculate heatwave days

In [ ]:
# Calculate heatwave days based on the criteia; If WBT exceeds 90th percentile or above for 3 consecutive days, then its considered as heatwave event.

# Ensure year, month, and day are properly converted to integers
for col in ['year', 'month', 'day']:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce').fillna(0).astype(int)

# Create the 'date' column
df_all['date'] = pd.to_datetime(df_all[['year', 'month', 'day']], errors='coerce')

# Ensure 'st_nm' is a string
df_all['st_nm'] = df_all['st_nm'].astype(str).fillna("Unknown_State")

# Ensure Wet_Bulb_Temperature_C is numeric
df_all['Wet_Bulb_Temperature_C'] = pd.to_numeric(df_all['Wet_Bulb_Temperature_C'], errors='coerce')

# Drop rows with missing temperature values
df_all = df_all.dropna(subset=['Wet_Bulb_Temperature_C'])

# Calculate the 90th percentile of Wet Bulb Temperature per state
percentile_90 = df_all.groupby('st_nm')['Wet_Bulb_Temperature_C'].quantile(0.90).rename("wbt_90th_percentile")

# Check if percentile_90 contains valid data
print("90th Percentile Data Sample:\n", percentile_90.head())

# Drop existing column before merging to avoid duplication issues
if "wbt_90th_percentile" in df_all.columns:
    df_all = df_all.drop(columns=["wbt_90th_percentile"])

# Merge the computed percentiles into df_all
df_all = df_all.merge(percentile_90, on="st_nm", how="left")

# Verify if 'wbt_90th_percentile' exists in df_all
print("Columns in df_all after merging:", df_all.columns)

# Flag heatwave days where WBT exceeds the 90th percentile
df_all['heatwave_flag'] = df_all['Wet_Bulb_Temperature_C'] >= df_all['wbt_90th_percentile']
df_all['heatwave_flag'] = df_all['heatwave_flag'].astype(int)  # Convert Boolean to int

# Sort by state and date before processing streaks
df_all = df_all.sort_values(by=['st_nm', 'date']).reset_index(drop=True)

# Identify consecutive heatwave streaks (at least 3 days)
df_all['streak'] = df_all.groupby('st_nm')['heatwave_flag'].apply(
    lambda x: x * (x.groupby((x == 0).cumsum()).cumcount() + 1)
).reset_index(drop=True)

# Flag heatwave events where streak is 3 or more consecutive days
df_all['heatwave_event'] = df_all['streak'] >= 3

# Count heatwave days per state and year
heatwave_stats = df_all[df_all['heatwave_event']].groupby(['year', 'st_nm']).agg(
    heatwave_days=('heatwave_event', 'sum'),
    median_wbt=('Wet_Bulb_Temperature_C', 'median')
).reset_index()

# Save the results
output_path = "/content/drive/MyDrive/ERA_5_INDIA_DATA_DIR/heatwave_summary.csv"
heatwave_stats.to_csv(output_path, index=False)